In [ ]:
from dapnn.imports import *
from dapnn.data_processing import *
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import random

import warnings
warnings.filterwarnings(action='once')

In [ ]:
# Set the random seed for reproducible results
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything()

keeping only the traces in the log that will create the normal model

In [ ]:
log_path='data/csv/PDC2020_training/'
log = import_log(log_path)

num_traces = len(log['trace_id'].unique())
ratio = 0.8
traces_for_normal_model = int(num_traces * ratio)

log = log[log['trace_id'].str.split(' ').str[1].astype(int) < traces_for_normal_model]
log.head(5)

In [ ]:
def training_dl(log,cat_names='activity',seed=45,ws=5,bs=32):
    categorify=Categorify()
    o=PPObj(log,procs=categorify,cat_names=cat_names,y_names=cat_names,splits=split_traces(log,test_seed=seed,validation_seed=seed))
    dls=o.get_dls(windows=partial(windows_fast,ws=ws),bs=bs)
    return o,dls,categorify

In [ ]:
class ControlFlowModel(torch.nn.Module) :
    def __init__(self, o) :
        super().__init__()
        hidden=25
        vocab_act=len(o.procs.categorify['activity'])
        emb_dim_act = int(np.sqrt(vocab_act))+1

        self.emb_act = nn.Embedding(vocab_act,emb_dim_act)
        
        self.lstm_act = nn.LSTM(emb_dim_act, hidden, batch_first=True, num_layers=2)

        self.linear_act = nn.Linear(hidden, vocab_act)

    def forward(self, xcat):
        xcat=xcat[:,0]
        x_act=xcat
        x_act = self.emb_act(x_act)
        x_act,_ = self.lstm_act(x_act)
        x_act = x_act[:,-1]
        x_act = self.linear_act(x_act)
        return x_act

In [ ]:
class HideOutput:
    'A utility function that hides all outputs in a context'
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


def training_loop(learn,epoch,print_output,lr_find,fixed_learning_rate=0.01):
    '''
    Basic training loop that uses learning rate finder and one cycle training. 
    See fastai docs for more information
    '''
    if lr_find:
        lr=np.median([learn.lr_find(show_plot=print_output)[0] for i in range(5)])
        learn.fit_one_cycle(epoch,float(lr))
    else: learn.fit(epoch,fixed_learning_rate)


def train_validate(dls,m,metrics=accuracy,loss=F.cross_entropy,epoch=20,print_output=True,model_dir=".",lr_find=True,
                   patience=5,min_delta=0.005,show_plot=True,store_path='tmp_switched',model_name='.model'):
    '''
    Trains a model on the training set with early stopping based on the validation loss.
    Afterwards, applies it to the test set.
    '''
    cbs = [
      EarlyStoppingCallback(monitor='valid_loss',min_delta=min_delta, patience=patience),
      SaveModelCallback(fname=model_name),
      ]
    learn=Learner(dls, m, path=store_path, model_dir=model_dir, loss_func=loss ,metrics=metrics,cbs=cbs)

    if print_output:
        training_loop(learn,epoch,show_plot,lr_find=lr_find)
        return learn.validate(dl=dls[2])
    else:
        with HideOutput(),learn.no_bar(),learn.no_logging():
            training_loop(learn,epoch,show_plot,lr_find=lr_find)
            return learn.validate(dl=dls[2])

In [ ]:
squeeze_cross_entropy = lambda x,y:F.cross_entropy(x,y[0])
squeeze_accuracy =lambda x,y:accuracy(x,y[0])

In [ ]:
o,dls,categorify = training_dl(log)
m = ControlFlowModel(o)
train_val = train_validate(dls,m,epoch=20,metrics=squeeze_accuracy,loss=squeeze_cross_entropy, show_plot=False)

In [ ]:
test_path = 'data/csv/PDC2020_ground_truth/'
test_log = import_log(test_path)
test_log[test_log['normal'] == False].head(5)

In [ ]:
def process_test(test_log,categorify,cat_names='activity'):
    o=PPObj(test_log,procs=categorify,cat_names=cat_names,y_names=cat_names,do_setup=False)
    o.process() # map to the same categories as in the training set
    return o


def predict_next_step(o,m,ws=5):
    wds,idx=partial(windows_fast,ws=ws)(o.xs, o.event_ids)
    res=(m(LongTensor(wds).cuda()))
    return res,idx


In [ ]:
def calc_if_anomaly(res,o,idx,k=6):
    # for each activity, it is an anomaly if its not in the top k most likely activities in the next step
    sm = nn.Softmax(dim=1)
    y = o.items['activity'].iloc[idx].values
    p = sm(res)
    if_anomaly = []
    for i in range(len(y)):
        if_anomaly.append(1 if y[i] not in p[i].argsort(descending=True)[:k] else 0)
    return np.array(if_anomaly)


def get_anomalies_if_anomaly(if_anomaly,o,idx,r=0.01):
    """If a trace has more than r of its activities as anomalies, it is an anomaly"""
    df = pd.DataFrame(columns=['if_anomaly'])
    df['if_anomaly'] = if_anomaly
    df['trace_id'] = o.items.iloc[idx]['trace_id'].values
    df['normal'] = o.items.iloc[idx]['normal'].values
    y_true = (df.loc[df.trace_id.drop_duplicates().index].normal==False).tolist()
    cases = df.loc[df.trace_id.drop_duplicates().index].trace_id.tolist()
    # for each trace, if more than r of its activities are anomalies, it is an anomaly

    anomalies = set(list(df.loc[df.groupby('trace_id')['if_anomaly'].transform('mean') > r]['trace_id']))
    y_pred=[case in anomalies for case in cases]
    return y_pred, y_true

In [ ]:
o = process_test(test_log,categorify)
nsp, idx = predict_next_step(o, m)

if_anomaly = calc_if_anomaly(nsp, o, idx)
y_pred, y_true = get_anomalies_if_anomaly(if_anomaly,o,idx)

In [ ]:
print(f'F1 score: {f1_score(y_true, y_pred)}')
print(f'Accuracy score: {accuracy_score(y_true, y_pred)}')
print(f'Precision score: {precision_score(y_true, y_pred)}')
print(f'Recall score: {recall_score(y_true, y_pred)}')